In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import sys
import gc

In [9]:
import datetime
import calendar
def week_of_month(tgtdate):
    tgtdate = tgtdate.to_pydatetime()

    days_this_month = calendar.mdays[tgtdate.month]
    for i in range(1, days_this_month):
        d = datetime.datetime(tgtdate.year, tgtdate.month, i)
        if d.day - d.weekday() > 0:
            startdate = d
            break
    # now we canuse the modulo 7 appraoch
    return (tgtdate - startdate).days //7 + 1

In [78]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        #else: df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [3]:
path_input = 'input/'
bonds = pd.read_csv(path_input + 'Isin.csv', parse_dates=['ActualMaturityDateKey', 'IssueDateKey'])
print(bonds.shape)
bonds.head()

(27411, 17)


,IsinIdx,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType
0,0,238,2038-12-31,2005-11-29,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.246002e+09,STEP CPN
1,1,238,2033-12-31,2005-11-29,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED
2,2,238,2033-12-31,2005-11-29,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.501245e+10,FIXED
3,3,236,2017-04-17,2007-04-17,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,B,Government,Sovereign,Domestic,7.340076e+09,FIXED
4,4,234,2022-10-04,2010-02-22,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,3.058452e+09,FLOATING


In [5]:
'''
rating_codes = {'A': 6, 'A+': 5, 'A-': 7, 'AA': 3, 'AA+': 2, 'AA-': 4, 'AAA': 1, 
                'B': 15, 'B+': 14, 'B-': 16, 'BB': 12, 'BB+': 11, 'BB-': 13, 'BBB': 9, 'BBB+': 8, 'BBB-': 10,
                'C': 24, 'C+': 23, 'CC': 21, 'CC+': 20, 'CC-': 22, 'CCC': 18, 'CCC+': 17, 'CCC-': 19, 
                'D': 28, 'DD+': 27, 'DDD': 26, 'DDD+': 25, 'NR': -999}
'''
rating_codes = {'A': 3, 'A+': 3, 'A-': 3, 'AA': 2, 'AA+': 2, 'AA-': 2, 'AAA': 1, 
                'B': 5, 'B+': 5, 'B-': 5, 'BB': 5, 'BB+': 5, 'BB-': 5, 'BBB': 4, 'BBB+': 4, 'BBB-': 4,
                'C': 6, 'C+': 6, 'CC': 6, 'CC+': 6, 'CC-': 6, 'CCC': 6, 'CCC+': 6, 'CCC-': 6, 
                'D': 7, 'DD+': 7, 'DDD': 7, 'DDD+': 7, 'NR': 10}

bonds['CompositeRating'] = bonds['CompositeRating'].map(rating_codes)
bonds['IssuedAmount'] = np.log1p(bonds.IssuedAmount)
bonds['Maturity_Time'] = bonds.ActualMaturityDateKey - bonds.IssueDateKey
bonds['Maturity_Time'] = bonds.Maturity_Time.astype('timedelta64[D]')/365
bonds['Issue_Year'] = bonds['IssueDateKey'].dt.year
bonds['Issue_Month'] = bonds['IssueDateKey'].dt.month
bonds['Issue_Week_of_Month'] = bonds['IssueDateKey'].apply(week_of_month)
bonds['Maturity_Year'] = bonds['ActualMaturityDateKey'].dt.year
bonds['Maturity_Month'] = bonds['ActualMaturityDateKey'].dt.month
bonds['Maturity_Week_of_Month'] = bonds['ActualMaturityDateKey'].apply(week_of_month)

fdrop = ['ActualMaturityDateKey', 'IssueDateKey', 'TickerIdx']
bonds.drop(fdrop, axis = 1, inplace = True)
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,MarketIssue,IssuedAmount,CouponType,Maturity_Time,Issue_Year,Issue_Month,Issue_Week_of_Month,Maturity_Year,Maturity_Month,Maturity_Week_of_Month
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,Domestic,20.943206,STEP CPN,33.109589,2005,11,4,2038,12,4
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,Domestic,22.312723,FIXED,28.106849,2005,11,4,2033,12,4
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,Domestic,23.432146,FIXED,28.106849,2005,11,4,2033,12,4
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,5,Government,...,Domestic,22.716615,FIXED,10.008219,2007,4,3,2017,4,3
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,Domestic,21.841175,FLOATING,12.621918,2010,2,4,2022,10,1


In [3]:
market = pd.read_csv(path_input + 'Market.csv', parse_dates=['DateKey'])
print(market.shape)
market.head()

(9867747, 5)


,IsinIdx,DateKey,Price,Yield,ZSpread
0,1,2016-01-01,104.250,7.835,5.505
1,7,2016-01-01,107.500,7.520,5.541
2,102,2016-01-01,100.746,4.048,2.085
3,331,2016-01-01,112.790,-0.752,-0.215
4,345,2016-01-01,113.383,-0.667,-0.272


In [5]:
market['Year'] = market['DateKey'].dt.year
market['Month'] = market['DateKey'].dt.month
market['Week_of_Month'] = market['DateKey'].apply(week_of_month)
market['TimeHash'] = market.Year*1000 + market.Month*10 + market.Week_of_Month
timehash_weekid = pd.read_csv(path_input + 'TimeHash-WeekID.csv', usecols=['TimeHash', 'WeekID'])
market = market.merge(timehash_weekid)
market['Prev_WeekID'] = market.WeekID-1
market.head()

,IsinIdx,DateKey,Price,Yield,ZSpread,Year,Month,Week_of_Month,TimeHash,WeekID,Prev_WeekID
0,1,2016-01-01,104.250,7.835,5.505,2016,1,0,2016010,1,0
1,7,2016-01-01,107.500,7.520,5.541,2016,1,0,2016010,1,0
2,102,2016-01-01,100.746,4.048,2.085,2016,1,0,2016010,1,0
3,331,2016-01-01,112.790,-0.752,-0.215,2016,1,0,2016010,1,0
4,345,2016-01-01,113.383,-0.667,-0.272,2016,1,0,2016010,1,0


In [6]:
market.isnull().sum()

IsinIdx          0
DateKey          0
Price            0
Yield            0
ZSpread          0
Year             0
Month            0
Week_of_Month    0
TimeHash         0
WeekID           0
Prev_WeekID      0
dtype: int64

In [7]:
print(market.WeekID.min())
print(market.WeekID.max())

1
140


In [8]:
market.drop_duplicates(['IsinIdx', 'WeekID']).shape

(2243685, 11)

In [10]:
grp = market.groupby(['IsinIdx', 'WeekID'])

temp = grp['Price', 'Yield', 'ZSpread'].mean().reset_index()
temp.columns = ['IsinIdx', 'WeekID', 'Avg_Price', 'Avg_Yield', 'Avg_ZSpread']

temp1 = grp['Price', 'Yield', 'ZSpread'].min().reset_index()
temp1.columns = ['IsinIdx', 'WeekID', 'Min_Price', 'Min_Yield', 'Min_ZSpread']
temp = temp.merge(temp1, how='left')

temp1 = grp['Price', 'Yield', 'ZSpread'].max().reset_index()
temp1.columns = ['IsinIdx', 'WeekID', 'Max_Price', 'Max_Yield', 'Max_ZSpread']
temp = temp.merge(temp1, how='left')

print(temp.shape)
temp.head()

(2243685, 11)


,IsinIdx,WeekID,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread
0,1,1,104.250,7.8350,5.5050,104.250,7.835,5.505,104.25,7.835,5.505
1,1,2,103.900,7.8706,5.6304,103.500,7.810,5.503,104.50,7.912,5.760
2,1,3,102.200,8.0458,5.9564,101.250,7.963,5.801,103.00,8.145,6.118
3,1,4,102.050,8.0614,6.0388,101.250,7.911,5.881,103.50,8.145,6.164
4,1,5,103.375,7.9232,5.9136,102.375,7.859,5.854,104.00,8.027,5.990


In [11]:
temp.describe()

,IsinIdx,WeekID,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread
count,2.243685e+06,2.243685e+06,2.243685e+06,2.243685e+06,2.243685e+06,2.243685e+06,2.243685e+06,2.243685e+06,2.243685e+06,2.243685e+06,2.243685e+06
mean,1.393772e+04,7.420206e+01,1.054720e+02,1.966293e+01,2.140264e+03,1.052314e+02,1.313409e+01,1.712207e+03,1.057098e+02,3.052911e+01,2.610645e+03
std,7.480199e+03,4.033482e+01,1.205878e+01,3.361159e+03,3.101176e+05,1.199922e+01,2.010231e+03,2.708732e+05,1.212770e+01,7.004945e+03,3.737261e+05
min,1.000000e+00,1.000000e+00,1.500000e-01,-3.893648e+03,-1.949085e+02,1.500000e-01,-1.170163e+04,-2.484130e+02,1.500000e-01,-1.460407e+03,-1.949040e+02
25%,7.534000e+03,4.000000e+01,1.002258e+02,1.156600e+00,2.930000e-01,1.001140e+02,1.119000e+00,2.700000e-01,1.003410e+02,1.194000e+00,3.140000e-01
50%,1.404300e+04,7.500000e+01,1.030035e+02,2.573200e+00,9.322000e-01,1.028310e+02,2.529000e+00,9.000000e-01,1.031960e+02,2.617000e+00,9.640000e-01
75%,2.076000e+04,1.100000e+02,1.084576e+02,3.900200e+00,1.939200e+00,1.082500e+02,3.854000e+00,1.892000e+00,1.086700e+02,3.948000e+00,1.987000e+00
max,2.734000e+04,1.400000e+02,2.123012e+02,2.038343e+06,1.115710e+08,2.114860e+02,1.537424e+06,1.036017e+08,2.144400e+02,5.301371e+06,1.411321e+08


In [12]:
market = market.merge(temp, how='left')
print(market.shape)
market.head()

(9867747, 20)


,IsinIdx,DateKey,Price,Yield,ZSpread,Year,Month,Week_of_Month,TimeHash,WeekID,Prev_WeekID,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread
0,1,2016-01-01,104.250,7.835,5.505,2016,1,0,2016010,1,0,104.250,7.835,5.505,104.250,7.835,5.505,104.250,7.835,5.505
1,7,2016-01-01,107.500,7.520,5.541,2016,1,0,2016010,1,0,107.500,7.520,5.541,107.500,7.520,5.541,107.500,7.520,5.541
2,102,2016-01-01,100.746,4.048,2.085,2016,1,0,2016010,1,0,100.746,4.048,2.085,100.746,4.048,2.085,100.746,4.048,2.085
3,331,2016-01-01,112.790,-0.752,-0.215,2016,1,0,2016010,1,0,112.790,-0.752,-0.215,112.790,-0.752,-0.215,112.790,-0.752,-0.215
4,345,2016-01-01,113.383,-0.667,-0.272,2016,1,0,2016010,1,0,113.383,-0.667,-0.272,113.383,-0.667,-0.272,113.383,-0.667,-0.272


In [13]:
del grp, temp, temp1; gc.collect()

489

In [15]:
fdrop = ['Price', 'Yield', 'ZSpread']
market.drop(fdrop, axis=1, inplace=True)

In [16]:
market = market.drop_duplicates(['IsinIdx', 'WeekID']).reset_index(drop=True)
print(market.shape)
market.head()

(2243685, 17)


,IsinIdx,DateKey,Year,Month,Week_of_Month,TimeHash,WeekID,Prev_WeekID,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread
0,1,2016-01-01,2016,1,0,2016010,1,0,104.250,7.835,5.505,104.250,7.835,5.505,104.250,7.835,5.505
1,7,2016-01-01,2016,1,0,2016010,1,0,107.500,7.520,5.541,107.500,7.520,5.541,107.500,7.520,5.541
2,102,2016-01-01,2016,1,0,2016010,1,0,100.746,4.048,2.085,100.746,4.048,2.085,100.746,4.048,2.085
3,331,2016-01-01,2016,1,0,2016010,1,0,112.790,-0.752,-0.215,112.790,-0.752,-0.215,112.790,-0.752,-0.215
4,345,2016-01-01,2016,1,0,2016010,1,0,113.383,-0.667,-0.272,113.383,-0.667,-0.272,113.383,-0.667,-0.272


In [22]:
market.to_csv(path_input+'market_weekly.csv', index=False)

In [6]:
bonds.to_csv(path_input+'bonds_updated.csv', index=False)

# Generating Features

In [2]:
full = pd.read_csv('input/generated/trade_challenge_final.csv')
print(full.shape)
full.head()

(7581011, 12)


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,NotionalEUR,PredictionIdx,Price,TimeHash,WeekID,Week_of_Month,Year
0,Sell,2789,1.0,8478,12,653168.0,train,0.0,2016121,57,1,2016
1,Sell,2398,1.0,9452,12,716873.0,train,0.0,2016121,57,1,2016
2,Sell,2398,1.0,12036,12,785588.0,train,0.0,2016121,57,1,2016
3,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016
4,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,1,2016


In [7]:
bonds = pd.read_csv('input/original/Isin.csv', parse_dates=['ActualMaturityDateKey', 'IssueDateKey'])
print(bonds.shape)
bonds.head()

(27411, 17)


,IsinIdx,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType
0,0,238,2038-12-31,2005-11-29,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.246002e+09,STEP CPN
1,1,238,2033-12-31,2005-11-29,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED
2,2,238,2033-12-31,2005-11-29,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.501245e+10,FIXED
3,3,236,2017-04-17,2007-04-17,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,B,Government,Sovereign,Domestic,7.340076e+09,FIXED
4,4,234,2022-10-04,2010-02-22,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,3.058452e+09,FLOATING


In [10]:
'''
rating_codes = {'A': 6, 'A+': 5, 'A-': 7, 'AA': 3, 'AA+': 2, 'AA-': 4, 'AAA': 1, 
                'B': 15, 'B+': 14, 'B-': 16, 'BB': 12, 'BB+': 11, 'BB-': 13, 'BBB': 9, 'BBB+': 8, 'BBB-': 10,
                'C': 24, 'C+': 23, 'CC': 21, 'CC+': 20, 'CC-': 22, 'CCC': 18, 'CCC+': 17, 'CCC-': 19, 
                'D': 28, 'DD+': 27, 'DDD': 26, 'DDD+': 25, 'NR': -999}
'''
rating_codes = {'A': 3, 'A+': 3, 'A-': 3, 'AA': 2, 'AA+': 2, 'AA-': 2, 'AAA': 1, 
                'B': 5, 'B+': 5, 'B-': 5, 'BB': 5, 'BB+': 5, 'BB-': 5, 'BBB': 4, 'BBB+': 4, 'BBB-': 4,
                'C': 6, 'C+': 6, 'CC': 6, 'CC+': 6, 'CC-': 6, 'CCC': 6, 'CCC+': 6, 'CCC-': 6, 
                'D': 7, 'DD+': 7, 'DDD': 7, 'DDD+': 7, 'NR': 10}

bonds['CompositeRating'] = bonds['CompositeRating'].map(rating_codes)
bonds['IssuedAmount'] = np.log1p(bonds.IssuedAmount)
bonds['Maturity_Time'] = bonds.ActualMaturityDateKey - bonds.IssueDateKey
bonds['Maturity_Time'] = bonds.Maturity_Time.astype('timedelta64[D]')/365
bonds['Issue_Year'] = bonds['IssueDateKey'].dt.year
bonds['Issue_Month'] = bonds['IssueDateKey'].dt.month
bonds['Issue_Week_of_Month'] = bonds['IssueDateKey'].apply(week_of_month)
bonds['Maturity_Year'] = bonds['ActualMaturityDateKey'].dt.year
bonds['Maturity_Month'] = bonds['ActualMaturityDateKey'].dt.month
bonds['Maturity_Week_of_Month'] = bonds['ActualMaturityDateKey'].apply(week_of_month)

fdrop = ['ActualMaturityDateKey', 'IssueDateKey', 'TickerIdx']
bonds.drop(fdrop, axis = 1, inplace = True)
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,MarketIssue,IssuedAmount,CouponType,Maturity_Time,Issue_Year,Issue_Month,Issue_Week_of_Month,Maturity_Year,Maturity_Month,Maturity_Week_of_Month
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,Domestic,3.088458,STEP CPN,33.109589,2005,11,4,2038,12,4
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,Domestic,3.148999,FIXED,28.106849,2005,11,4,2033,12,4
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,Domestic,3.195900,FIXED,28.106849,2005,11,4,2033,12,4
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,Domestic,3.166176,FIXED,10.008219,2007,4,3,2017,4,3
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,Domestic,3.128565,FLOATING,12.621918,2010,2,4,2022,10,1


In [11]:
# Total number of time bond is bought/sold
feat = ['CustomerIdx', 'IsinIdx']
temp = full[(full.CustomerInterest == 1) & (full.PredictionIdx !='valid')][feat].groupby(['IsinIdx']).size().reset_index()
temp.columns = ['IsinIdx', 'nBondsTimes']
temp['nBondsTimes'].fillna(0, inplace=True)
bonds = bonds.merge(temp, how='left', on='IsinIdx')
print(bonds.shape)
del temp; gc.collect()
bonds.head()

(27411, 22)


,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,IssuedAmount,CouponType,Maturity_Time,Issue_Year,Issue_Month,Issue_Week_of_Month,Maturity_Year,Maturity_Month,Maturity_Week_of_Month,nBondsTimes
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,3.088458,STEP CPN,33.109589,2005,11,4,2038,12,4,32.0
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,3.148999,FIXED,28.106849,2005,11,4,2033,12,4,563.0
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,3.195900,FIXED,28.106849,2005,11,4,2033,12,4,11.0
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,3.166176,FIXED,10.008219,2007,4,3,2017,4,3,1282.0
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,3.128565,FLOATING,12.621918,2010,2,4,2022,10,1,3.0


In [12]:
# No unique customer buys a bond
feat = ['CustomerIdx', 'IsinIdx']
temp = full[(full.CustomerInterest == 1) & (full.PredictionIdx !='valid')][feat].drop_duplicates().groupby(['IsinIdx']).size().reset_index()
temp.columns = ['IsinIdx', 'nBondsUniqueCust']
temp['nBondsUniqueCust'].fillna(0, inplace=True)
bonds = bonds.merge(temp, how='left', on='IsinIdx')
print(bonds.shape)
del temp; gc.collect()
bonds.head()

(27411, 23)


,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,CouponType,Maturity_Time,Issue_Year,Issue_Month,Issue_Week_of_Month,Maturity_Year,Maturity_Month,Maturity_Week_of_Month,nBondsTimes,nBondsUniqueCust
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,STEP CPN,33.109589,2005,11,4,2038,12,4,32.0,14.0
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,FIXED,28.106849,2005,11,4,2033,12,4,563.0,44.0
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,FIXED,28.106849,2005,11,4,2033,12,4,11.0,5.0
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,FIXED,10.008219,2007,4,3,2017,4,3,1282.0,117.0
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,FLOATING,12.621918,2010,2,4,2022,10,1,3.0,1.0


In [16]:
# Bond reBuying ratio
bonds['reBuying_ratio'] = bonds.nBondsTimes/bonds.nBondsUniqueCust
bonds['reBuying_ratio'].fillna(0, inplace=True)
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,Maturity_Time,Issue_Year,Issue_Month,Issue_Week_of_Month,Maturity_Year,Maturity_Month,Maturity_Week_of_Month,nBondsTimes,nBondsUniqueCust,reBuying_ratio
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,33.109589,2005,11,4,2038,12,4,32.0,14.0,2.285714
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,28.106849,2005,11,4,2033,12,4,563.0,44.0,12.795455
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,28.106849,2005,11,4,2033,12,4,11.0,5.0,2.200000
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,10.008219,2007,4,3,2017,4,3,1282.0,117.0,10.957265
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,12.621918,2010,2,4,2022,10,1,3.0,1.0,3.000000


In [17]:
bonds.nBondsTimes.describe()

count    27169.000000
mean        85.835806
std        120.462899
min          1.000000
25%         14.000000
50%         46.000000
75%        111.000000
max       2543.000000
Name: nBondsTimes, dtype: float64

In [18]:
bonds.nBondsTimes.sum()

2332073.0

In [20]:
# Probability(times 100) of bond being bought: nTimes/nbondsTimes*10000
bonds['bond_probability'] = (bonds.nBondsTimes/2332073.0)*10000
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,Issue_Year,Issue_Month,Issue_Week_of_Month,Maturity_Year,Maturity_Month,Maturity_Week_of_Month,nBondsTimes,nBondsUniqueCust,reBuying_ratio,bond_probability
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,2005,11,4,2038,12,4,32.0,14.0,2.285714,0.137217
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,2005,11,4,2033,12,4,563.0,44.0,12.795455,2.414161
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,2005,11,4,2033,12,4,11.0,5.0,2.200000,0.047168
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,2007,4,3,2017,4,3,1282.0,117.0,10.957265,5.497255
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,2010,2,4,2022,10,1,3.0,1.0,3.000000,0.012864


In [22]:
# nBondTimes for week 0,1,2,3,4,5
feat = ['CustomerIdx', 'IsinIdx', 'Week_of_Month']
temp = full[(full.CustomerInterest == 1) & (full.PredictionIdx !='valid')][feat].groupby(['IsinIdx', 'Week_of_Month']).size().reset_index()
temp.columns = ['IsinIdx', 'Week_of_Month', 'nBondsTimes']
temp['nBondsTimes'].fillna(0, inplace=True)
temp = temp.pivot(index='IsinIdx', columns='Week_of_Month', values='nBondsTimes').reset_index()
temp.columns = ['IsinIdx', 'nBondTimes_Week0', 'nBondTimes_Week1', 'nBondTimes_Week2', 'nBondTimes_Week3', 
                'nBondTimes_Week4', 'nBondTimes_Week5']
temp.fillna(0, inplace=True)
bonds = bonds.merge(temp, how='left')
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,nBondsTimes,nBondsUniqueCust,reBuying_ratio,bond_probability,nBondTimes_Week0,nBondTimes_Week1,nBondTimes_Week2,nBondTimes_Week3,nBondTimes_Week4,nBondTimes_Week5
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,32.0,14.0,2.285714,0.137217,1.0,5.0,9.0,3.0,14.0,0.0
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,563.0,44.0,12.795455,2.414161,43.0,130.0,123.0,133.0,119.0,15.0
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,11.0,5.0,2.200000,0.047168,0.0,0.0,4.0,0.0,7.0,0.0
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,1282.0,117.0,10.957265,5.497255,113.0,310.0,319.0,269.0,247.0,24.0
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,3.0,1.0,3.000000,0.012864,0.0,2.0,1.0,0.0,0.0,0.0


In [23]:
# Probability of bond for week 1,2,3,4
bonds['bond_probability_week0'] = (bonds.nBondTimes_Week0/bonds.nBondsTimes)*bonds.bond_probability
bonds['bond_probability_week1'] = (bonds.nBondTimes_Week1/bonds.nBondsTimes)*bonds.bond_probability
bonds['bond_probability_week2'] = (bonds.nBondTimes_Week2/bonds.nBondsTimes)*bonds.bond_probability
bonds['bond_probability_week3'] = (bonds.nBondTimes_Week3/bonds.nBondsTimes)*bonds.bond_probability
bonds['bond_probability_week4'] = (bonds.nBondTimes_Week4/bonds.nBondsTimes)*bonds.bond_probability
bonds['bond_probability_week5'] = (bonds.nBondTimes_Week5/bonds.nBondsTimes)*bonds.bond_probability
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,nBondTimes_Week2,nBondTimes_Week3,nBondTimes_Week4,nBondTimes_Week5,bond_probability_week0,bond_probability_week1,bond_probability_week2,bond_probability_week3,bond_probability_week4,bond_probability_week5
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,9.0,3.0,14.0,0.0,0.004288,0.021440,0.038592,0.012864,0.060032,0.000000
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,123.0,133.0,119.0,15.0,0.184385,0.557444,0.527428,0.570308,0.510276,0.064320
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,4.0,0.0,7.0,0.0,0.000000,0.000000,0.017152,0.000000,0.030016,0.000000
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,319.0,269.0,247.0,24.0,0.484547,1.329289,1.367882,1.153480,1.059144,0.102913
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,1.0,0.0,0.0,0.0,0.000000,0.008576,0.004288,0.000000,0.000000,0.000000


In [33]:
# Total number of time bond is bought/sold
feat = ['CustomerIdx', 'IsinIdx', 'BuySell']
temp = full[(full.CustomerInterest == 1) & (full.PredictionIdx !='valid')][feat].groupby(['IsinIdx', 'BuySell']).size().reset_index()
temp.columns = ['IsinIdx','BuySell', 'nBondsTimes']
temp = temp.pivot(index='IsinIdx', columns='BuySell', values='nBondsTimes').reset_index().head()
temp.columns = ['IsinIdx','Buy_nBondsTimes', 'Sell_nBondsTimes']
temp.fillna(0, inplace=True)
temp['nBondsTimes'] = temp['Buy_nBondsTimes'] + temp['Sell_nBondsTimes']
temp['bond_prob_buy'] = temp['Buy_nBondsTimes']/temp['nBondsTimes']
temp['bond_prob_sell'] = temp['Sell_nBondsTimes']/temp['nBondsTimes']
temp.fillna(0, inplace=True)
temp.drop('nBondsTimes', axis=1, inplace=True)
bonds = bonds.merge(temp, how='left')
del temp, feat; gc.collect()
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,bond_probability_week0,bond_probability_week1,bond_probability_week2,bond_probability_week3,bond_probability_week4,bond_probability_week5,Buy_nBondsTimes,Sell_nBondsTimes,bond_prob_buy,bond_prob_sell
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,0.004288,0.021440,0.038592,0.012864,0.060032,0.000000,11.0,21.0,0.343750,0.656250
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,0.184385,0.557444,0.527428,0.570308,0.510276,0.064320,323.0,240.0,0.573712,0.426288
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,0.000000,0.000000,0.017152,0.000000,0.030016,0.000000,4.0,7.0,0.363636,0.636364
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,0.484547,1.329289,1.367882,1.153480,1.059144,0.102913,821.0,461.0,0.640406,0.359594
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,0.000000,0.008576,0.004288,0.000000,0.000000,0.000000,2.0,1.0,0.666667,0.333333


In [34]:
market = pd.read_csv('input/generated/market_weekly.csv', parse_dates = ['DateKey'])
fdrop = ['DateKey', 'Prev_WeekID']
market.drop(fdrop, axis=1, inplace=True)
print(market.shape)
market.head()

(2243685, 15)


,IsinIdx,Year,Month,Week_of_Month,TimeHash,WeekID,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread
0,1,2016,1,0,2016010,1,104.250,7.835,5.505,104.250,7.835,5.505,104.250,7.835,5.505
1,7,2016,1,0,2016010,1,107.500,7.520,5.541,107.500,7.520,5.541,107.500,7.520,5.541
2,102,2016,1,0,2016010,1,100.746,4.048,2.085,100.746,4.048,2.085,100.746,4.048,2.085
3,331,2016,1,0,2016010,1,112.790,-0.752,-0.215,112.790,-0.752,-0.215,112.790,-0.752,-0.215
4,345,2016,1,0,2016010,1,113.383,-0.667,-0.272,113.383,-0.667,-0.272,113.383,-0.667,-0.272


In [58]:
temp = full.merge(market, how='left')
feat = ['BuySell', 'IsinIdx', 'Avg_Price', 'Avg_Yield', 'Avg_ZSpread']
temp = temp.drop_duplicates(['BuySell', 'IsinIdx', 'WeekID'])[feat].groupby(['BuySell', 'IsinIdx']).mean().reset_index()
temp = temp.pivot_table(index='IsinIdx', columns='BuySell', values=['Avg_Price', 'Avg_Yield', 'Avg_ZSpread']).reset_index().head(20)
temp.columns=['IsinIdx', 'Bond_Buy_Avg_Price', 'Bond_Sell_Avg_Price', 'Bond_Buy_Avg_Yield',
              'Bond_Sell_Avg_Yield', 'Bond_Buy_Avg_ZSpread', 'Bond_Sell_Avg_ZSpread']
bonds = bonds.merge(temp, how='left')
del temp, feat; gc.collect()
print(bonds.shape)
bonds.head()

(27411, 47)


,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,Buy_nBondsTimes,Sell_nBondsTimes,bond_prob_buy,bond_prob_sell,Bond_Buy_Avg_Price,Bond_Sell_Avg_Price,Bond_Buy_Avg_Yield,Bond_Sell_Avg_Yield,Bond_Buy_Avg_ZSpread,Bond_Sell_Avg_ZSpread
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,11.0,21.0,0.343750,0.656250,NaN,NaN,NaN,NaN,NaN,NaN
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,323.0,240.0,0.573712,0.426288,110.38334,110.405267,7.211648,7.208253,5.032488,5.004804
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,4.0,7.0,0.363636,0.636364,NaN,NaN,NaN,NaN,NaN,NaN
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,821.0,461.0,0.640406,0.359594,NaN,NaN,NaN,NaN,NaN,NaN
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NaN,Government,...,2.0,1.0,0.666667,0.333333,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
bonds.to_csv('input/generated/bonds_feat.csv', index=False)

In [60]:
full.isnull().sum()

BuySell                   0
CustomerIdx               0
CustomerInterest     484758
IsinIdx                   0
Month                     0
NotionalEUR         5207872
PredictionIdx             0
Price               5270744
TimeHash                  0
WeekID                    0
Week_of_Month             0
Year                      0
dtype: int64

In [61]:
full = full.merge(market, how='left')
print(full.shape)
full.head()

(7581011, 21)


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,NotionalEUR,PredictionIdx,Price,TimeHash,WeekID,...,Year,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread
0,Sell,2789,1.0,8478,12,653168.0,train,0.0,2016121,57,...,2016,86.4376,8.8726,6.7894,85.677,8.801,6.702,86.812,9.017,6.946
1,Sell,2398,1.0,9452,12,716873.0,train,0.0,2016121,57,...,2016,98.3128,5.6364,3.5346,97.062,5.331,3.162,100.281,5.832,3.743
2,Sell,2398,1.0,12036,12,785588.0,train,0.0,2016121,57,...,2016,102.8648,4.0736,2.4798,102.125,3.973,2.360,103.188,4.311,2.717
3,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,...,2016,100.1488,1.1422,0.1536,100.129,1.048,0.077,100.168,1.213,0.198
4,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,...,2016,100.1488,1.1422,0.1536,100.129,1.048,0.077,100.168,1.213,0.198


In [62]:
full.isnull().sum()

BuySell                   0
CustomerIdx               0
CustomerInterest     484758
IsinIdx                   0
Month                     0
NotionalEUR         5207872
PredictionIdx             0
Price               5270744
TimeHash                  0
WeekID                    0
Week_of_Month             0
Year                      0
Avg_Price           1850235
Avg_Yield           1850235
Avg_ZSpread         1850235
Min_Price           1850235
Min_Yield           1850235
Min_ZSpread         1850235
Max_Price           1850235
Max_Yield           1850235
Max_ZSpread         1850235
dtype: int64

In [63]:
full['Prev_WeekID'] = full.WeekID-1

In [64]:
full.WeekID.max()

141

In [65]:
# Unique Bonds Last week
feat = ['CustomerIdx', 'IsinIdx', 'WeekID', 'Prev_WeekID']
temp = full[(full.CustomerInterest != 0) & (full.PredictionIdx != 'valid')][feat].drop_duplicates().groupby(['CustomerIdx', 'WeekID', 'Prev_WeekID']).size().reset_index()
temp.columns = ['CustomerIdx', 'WeekID', 'Prev_WeekID', 'nUniqueBonds_lastWeek']
temp = temp[['CustomerIdx', 'Prev_WeekID']].merge(temp[['CustomerIdx', 'WeekID', 'nUniqueBonds_lastWeek']], left_on=['CustomerIdx', 'Prev_WeekID'], right_on=['CustomerIdx', 'WeekID'])
temp.drop('WeekID', axis=1, inplace=True)
full = full.merge(temp, how='left')
full['nUniqueBonds_lastWeek'].fillna(0, inplace=True)
del temp; gc.collect()
print(full.shape)
full.head()

(7581011, 23)


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,NotionalEUR,PredictionIdx,Price,TimeHash,WeekID,...,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread,Prev_WeekID,nUniqueBonds_lastWeek
0,Sell,2789,1.0,8478,12,653168.0,train,0.0,2016121,57,...,8.8726,6.7894,85.677,8.801,6.702,86.812,9.017,6.946,56,11.0
1,Sell,2398,1.0,9452,12,716873.0,train,0.0,2016121,57,...,5.6364,3.5346,97.062,5.331,3.162,100.281,5.832,3.743,56,92.0
2,Sell,2398,1.0,12036,12,785588.0,train,0.0,2016121,57,...,4.0736,2.4798,102.125,3.973,2.360,103.188,4.311,2.717,56,92.0
3,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,...,1.1422,0.1536,100.129,1.048,0.077,100.168,1.213,0.198,56,310.0
4,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,...,1.1422,0.1536,100.129,1.048,0.077,100.168,1.213,0.198,56,310.0


In [66]:
full.columns

Index(['BuySell', 'CustomerIdx', 'CustomerInterest', 'IsinIdx', 'Month',
       'NotionalEUR', 'PredictionIdx', 'Price', 'TimeHash', 'WeekID',
       'Week_of_Month', 'Year', 'Avg_Price', 'Avg_Yield', 'Avg_ZSpread',
       'Min_Price', 'Min_Yield', 'Min_ZSpread', 'Max_Price', 'Max_Yield',
       'Max_ZSpread', 'Prev_WeekID', 'nUniqueBonds_lastWeek'],
      dtype='object')

In [67]:
market.columns

Index(['IsinIdx', 'Year', 'Month', 'Week_of_Month', 'TimeHash', 'WeekID',
       'Avg_Price', 'Avg_Yield', 'Avg_ZSpread', 'Min_Price', 'Min_Yield',
       'Min_ZSpread', 'Max_Price', 'Max_Yield', 'Max_ZSpread'],
      dtype='object')

In [68]:
# Shifting price by 1 week
lfeat = ['BuySell', 'CustomerIdx', 'IsinIdx','Prev_WeekID']
rfeat = ['IsinIdx', 'WeekID', 'Avg_Price', 'Avg_Yield', 'Avg_ZSpread',
         'Min_Price', 'Min_Yield', 'Min_ZSpread', 'Max_Price', 'Max_Yield', 'Max_ZSpread']
lon = ['IsinIdx', 'Prev_WeekID']
ron = ['IsinIdx', 'WeekID']

temp = full[lfeat].merge(market[rfeat], how='left', left_on=lon, right_on=ron)
temp.columns = ['BuySell', 'CustomerIdx', 'IsinIdx', 'Prev_WeekID', 'WeekID',
       'Prev_Avg_Price', 'Prev_Avg_Yield', 'Prev_Avg_ZSpread', 'Prev_Min_Price', 'Prev_Min_Yield',
       'Prev_Min_ZSpread', 'Prev_Max_Price', 'Prev_Max_Yield', 'Prev_Max_ZSpread']
temp['WeekID'] = temp['Prev_WeekID']+1
temp.head()

,BuySell,CustomerIdx,IsinIdx,Prev_WeekID,WeekID,Prev_Avg_Price,Prev_Avg_Yield,Prev_Avg_ZSpread,Prev_Min_Price,Prev_Min_Yield,Prev_Min_ZSpread,Prev_Max_Price,Prev_Max_Yield,Prev_Max_ZSpread
0,Sell,2789,8478,56,56.0,85.7365,9.0050,6.9140,85.661,8.990,6.872,85.812,9.020,6.956
1,Sell,2398,9452,56,56.0,97.0565,5.8330,3.7230,97.051,5.832,3.694,97.062,5.834,3.752
2,Sell,2398,12036,56,56.0,102.1340,4.3085,2.7195,102.125,4.306,2.694,102.143,4.311,2.745
3,Sell,2574,7762,56,56.0,100.1770,1.0895,0.1310,100.173,1.071,0.115,100.181,1.108,0.147
4,Sell,2574,7762,56,56.0,100.1770,1.0895,0.1310,100.173,1.071,0.115,100.181,1.108,0.147


In [69]:
temp.isnull().sum()

BuySell                   0
CustomerIdx               0
IsinIdx                   0
Prev_WeekID               0
WeekID              1539118
Prev_Avg_Price      1539118
Prev_Avg_Yield      1539118
Prev_Avg_ZSpread    1539118
Prev_Min_Price      1539118
Prev_Min_Yield      1539118
Prev_Min_ZSpread    1539118
Prev_Max_Price      1539118
Prev_Max_Yield      1539118
Prev_Max_ZSpread    1539118
dtype: int64

In [81]:
feat = ['BuySell', 'CustomerIdx', 'CustomerInterest', 'IsinIdx', 'Month',
       'NotionalEUR', 'PredictionIdx', 'Price', 'TimeHash', 'WeekID',
       'Week_of_Month', 'Year', 'nUniqueBonds_lastWeek']
full = full[feat].merge(temp.drop_duplicates(), how='left', on=['BuySell', 'CustomerIdx','IsinIdx', 'WeekID'])
del temp, feat; gc.collect()
print(full.shape)
print(full.columns)
full.head()

(7581011, 23)
Index(['BuySell', 'CustomerIdx', 'CustomerInterest', 'IsinIdx', 'Month',
       'NotionalEUR', 'PredictionIdx', 'Price', 'TimeHash', 'WeekID',
       'Week_of_Month', 'Year', 'nUniqueBonds_lastWeek', 'Prev_WeekID',
       'Prev_Avg_Price', 'Prev_Avg_Yield', 'Prev_Avg_ZSpread',
       'Prev_Min_Price', 'Prev_Min_Yield', 'Prev_Min_ZSpread',
       'Prev_Max_Price', 'Prev_Max_Yield', 'Prev_Max_ZSpread'],
      dtype='object')


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,NotionalEUR,PredictionIdx,Price,TimeHash,WeekID,...,Prev_WeekID,Prev_Avg_Price,Prev_Avg_Yield,Prev_Avg_ZSpread,Prev_Min_Price,Prev_Min_Yield,Prev_Min_ZSpread,Prev_Max_Price,Prev_Max_Yield,Prev_Max_ZSpread
0,Sell,2789,1.0,8478,12,653168.0,train,0.0,2016121,57,...,56,85.7365,9.0050,6.9140,85.661,8.990,6.872,85.812,9.020,6.956
1,Sell,2398,1.0,9452,12,716873.0,train,0.0,2016121,57,...,56,97.0565,5.8330,3.7230,97.051,5.832,3.694,97.062,5.834,3.752
2,Sell,2398,1.0,12036,12,785588.0,train,0.0,2016121,57,...,56,102.1340,4.3085,2.7195,102.125,4.306,2.694,102.143,4.311,2.745
3,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,...,56,100.1770,1.0895,0.1310,100.173,1.071,0.115,100.181,1.108,0.147
4,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,...,56,100.1770,1.0895,0.1310,100.173,1.071,0.115,100.181,1.108,0.147


In [82]:
full.isnull().sum()

BuySell                        0
CustomerIdx                    0
CustomerInterest          484758
IsinIdx                        0
Month                          0
NotionalEUR              5207872
PredictionIdx                  0
Price                    5270744
TimeHash                       0
WeekID                         0
Week_of_Month                  0
Year                           0
nUniqueBonds_lastWeek          0
Prev_WeekID                    0
Prev_Avg_Price           1539118
Prev_Avg_Yield           1539118
Prev_Avg_ZSpread         1539118
Prev_Min_Price           1539118
Prev_Min_Yield           1539118
Prev_Min_ZSpread         1539118
Prev_Max_Price           1539118
Prev_Max_Yield           1539118
Prev_Max_ZSpread         1539118
dtype: int64

In [83]:
full.columns

Index(['BuySell', 'CustomerIdx', 'CustomerInterest', 'IsinIdx', 'Month',
       'NotionalEUR', 'PredictionIdx', 'Price', 'TimeHash', 'WeekID',
       'Week_of_Month', 'Year', 'nUniqueBonds_lastWeek', 'Prev_WeekID',
       'Prev_Avg_Price', 'Prev_Avg_Yield', 'Prev_Avg_ZSpread',
       'Prev_Min_Price', 'Prev_Min_Yield', 'Prev_Min_ZSpread',
       'Prev_Max_Price', 'Prev_Max_Yield', 'Prev_Max_ZSpread'],
      dtype='object')

In [84]:
feat = ['BuySell', 'CustomerIdx', 'IsinIdx', 'PredictionIdx', 'WeekID', 'nUniqueBonds_lastWeek',
       'Prev_Avg_Price', 'Prev_Avg_Yield', 'Prev_Avg_ZSpread', 'Prev_Min_Price', 'Prev_Min_Yield', 'Prev_Min_ZSpread',
       'Prev_Max_Price', 'Prev_Max_Yield', 'Prev_Max_ZSpread', 'bond_bought_nTimes_lastWeek', 'bond_sold_nTimes_lastWeek']
full[feat].to_csv('input/generated/bond_week_feat.csv', index=False)

In [96]:
# No of times this bond is bought & sold last week
feat = ['CustomerIdx', 'BuySell', 'IsinIdx', 'WeekID', 'Prev_WeekID']
temp = full[(full.CustomerInterest != 0) & (full.PredictionIdx !='valid')][feat].groupby(['IsinIdx', 'BuySell', 'WeekID', 'Prev_WeekID']).size().reset_index()
temp.columns = ['IsinIdx', 'BuySell', 'WeekID', 'Prev_WeekID', 'bond_nTimes_lastWeek']
temp = temp[['IsinIdx', 'BuySell', 'Prev_WeekID']].merge(temp[['IsinIdx', 'BuySell', 'WeekID', 'bond_nTimes_lastWeek']], left_on=['IsinIdx', 'BuySell', 'Prev_WeekID'], right_on=['IsinIdx', 'BuySell', 'WeekID'])
temp.drop('WeekID', axis=1, inplace=True)
temp = temp.pivot_table(index=['IsinIdx', 'Prev_WeekID'], columns='BuySell', values='bond_nTimes_lastWeek').reset_index().fillna(0)
temp.columns = ['IsinIdx', 'Prev_WeekID', 'bond_bought_nTimes_lastWeek', 'bond_sold_nTimes_lastWeek']
full = full.merge(temp, how='left')
full['bond_bought_nTimes_lastWeek'].fillna(0, inplace=True)
full['bond_sold_nTimes_lastWeek'].fillna(0, inplace=True)
del temp; gc.collect()
print(full.shape)
full.head()

(7581011, 25)


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,NotionalEUR,PredictionIdx,Price,TimeHash,WeekID,...,Prev_Avg_Yield,Prev_Avg_ZSpread,Prev_Min_Price,Prev_Min_Yield,Prev_Min_ZSpread,Prev_Max_Price,Prev_Max_Yield,Prev_Max_ZSpread,bond_bought_nTimes_lastWeek,bond_sold_nTimes_lastWeek
0,Sell,2789,1.0,8478,12,653168.0,train,0.0,2016121,57,...,9.0050,6.9140,85.661,8.990,6.872,85.812,9.020,6.956,0.0,0.0
1,Sell,2398,1.0,9452,12,716873.0,train,0.0,2016121,57,...,5.8330,3.7230,97.051,5.832,3.694,97.062,5.834,3.752,1.0,0.0
2,Sell,2398,1.0,12036,12,785588.0,train,0.0,2016121,57,...,4.3085,2.7195,102.125,4.306,2.694,102.143,4.311,2.745,0.0,1.0
3,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,...,1.0895,0.1310,100.173,1.071,0.115,100.181,1.108,0.147,0.0,1.0
4,Sell,2574,1.0,7762,12,513203.0,train,0.0,2016121,57,...,1.0895,0.1310,100.173,1.071,0.115,100.181,1.108,0.147,0.0,1.0


In [98]:
full.columns

Index(['BuySell', 'CustomerIdx', 'CustomerInterest', 'IsinIdx', 'Month',
       'NotionalEUR', 'PredictionIdx', 'Price', 'TimeHash', 'WeekID',
       'Week_of_Month', 'Year', 'nUniqueBonds_lastWeek', 'Prev_WeekID',
       'Prev_Avg_Price', 'Prev_Avg_Yield', 'Prev_Avg_ZSpread',
       'Prev_Min_Price', 'Prev_Min_Yield', 'Prev_Min_ZSpread',
       'Prev_Max_Price', 'Prev_Max_Yield', 'Prev_Max_ZSpread',
       'bond_bought_nTimes_lastWeek', 'bond_sold_nTimes_lastWeek'],
      dtype='object')

In [99]:
feat = ['BuySell', 'CustomerIdx', 'IsinIdx', 'PredictionIdx', 'WeekID', 'nUniqueBonds_lastWeek',
       'Prev_Avg_Price', 'Prev_Avg_Yield', 'Prev_Avg_ZSpread', 'Prev_Min_Price', 'Prev_Min_Yield', 'Prev_Min_ZSpread',
       'Prev_Max_Price', 'Prev_Max_Yield', 'Prev_Max_ZSpread', 'bond_bought_nTimes_lastWeek', 'bond_sold_nTimes_lastWeek']
full[feat].to_csv('input/generated/bond_week_feat.csv', index=False)

In [18]:
market = pd.read_csv(path_input+'market_weekly.csv', parse_dates = ['DateKey'])
feat = ['BuySell', 'CustomerIdx', 'CustomerInterest', 'IsinIdx', 'PredictionIdx', 'Year', 'Month', 'Week_of_Month',
        'Prev_WeekID', 'TimeHash', 'WeekID', 'Cust_Previous_WeekID']
full = pd.read_csv(path_input+'trade_challenge_custFeat.csv', usecols=feat)
print(market.shape)
print(full.shape)

C:\Users\shubh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2243685, 17)
(4934349, 12)


In [19]:
full.head()

,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,PredictionIdx,Prev_WeekID,TimeHash,WeekID,Week_of_Month,Year,Cust_Previous_WeekID
0,Sell,2789,1.0,8478,12,trade,56,2016121,57,1,2016,56.0
1,Sell,2398,1.0,9452,12,trade,56,2016121,57,1,2016,56.0
2,Sell,2398,1.0,12036,12,trade,56,2016121,57,1,2016,56.0
3,Buy,2574,1.0,6921,12,trade,56,2016121,57,1,2016,56.0
4,Sell,2574,1.0,7762,12,trade,56,2016121,57,1,2016,56.0


In [21]:
market.columns

Index(['IsinIdx', 'DateKey', 'Year', 'Month', 'Week_of_Month', 'TimeHash',
       'WeekID', 'Prev_WeekID', 'Avg_Price', 'Avg_Yield', 'Avg_ZSpread',
       'Min_Price', 'Min_Yield', 'Min_ZSpread', 'Max_Price', 'Max_Yield',
       'Max_ZSpread'],
      dtype='object')

In [22]:
market.drop(['DateKey', 'Year', 'Month', 'Week_of_Month'], axis=1, inplace=True)
market.head()

,IsinIdx,TimeHash,WeekID,Prev_WeekID,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread
0,1,2016010,1,0,104.250,7.835,5.505,104.250,7.835,5.505,104.250,7.835,5.505
1,7,2016010,1,0,107.500,7.520,5.541,107.500,7.520,5.541,107.500,7.520,5.541
2,102,2016010,1,0,100.746,4.048,2.085,100.746,4.048,2.085,100.746,4.048,2.085
3,331,2016010,1,0,112.790,-0.752,-0.215,112.790,-0.752,-0.215,112.790,-0.752,-0.215
4,345,2016010,1,0,113.383,-0.667,-0.272,113.383,-0.667,-0.272,113.383,-0.667,-0.272


In [23]:
full = full.merge(market, how='left')
print(full.shape)
full.head()

(4934349, 21)


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,PredictionIdx,Prev_WeekID,TimeHash,WeekID,Week_of_Month,...,Cust_Previous_WeekID,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread
0,Sell,2789,1.0,8478,12,trade,56,2016121,57,1,...,56.0,86.4376,8.8726,6.7894,85.677,8.801,6.702,86.812,9.017,6.946
1,Sell,2398,1.0,9452,12,trade,56,2016121,57,1,...,56.0,98.3128,5.6364,3.5346,97.062,5.331,3.162,100.281,5.832,3.743
2,Sell,2398,1.0,12036,12,trade,56,2016121,57,1,...,56.0,102.8648,4.0736,2.4798,102.125,3.973,2.360,103.188,4.311,2.717
3,Buy,2574,1.0,6921,12,trade,56,2016121,57,1,...,56.0,100.3772,6.9206,4.9414,99.375,6.852,4.842,100.750,7.099,5.017
4,Sell,2574,1.0,7762,12,trade,56,2016121,57,1,...,56.0,100.1488,1.1422,0.1536,100.129,1.048,0.077,100.168,1.213,0.198


In [24]:
full.isnull().sum()

BuySell                 2307930
CustomerIdx                   0
CustomerInterest         484758
IsinIdx                       0
Month                         0
PredictionIdx                 0
Prev_WeekID                   0
TimeHash                      0
WeekID                        0
Week_of_Month                 0
Year                          0
Cust_Previous_WeekID          0
Avg_Price               1235284
Avg_Yield               1235284
Avg_ZSpread             1235284
Min_Price               1235284
Min_Yield               1235284
Min_ZSpread             1235284
Max_Price               1235284
Max_Yield               1235284
Max_ZSpread             1235284
dtype: int64

In [26]:
# Total number of time bond is bought/sold
feat = ['CustomerIdx', 'IsinIdx']
temp = full[full.CustomerInterest == 1][feat].groupby(['IsinIdx']).size().reset_index()
temp.columns = ['IsinIdx', 'nBondsTimes']
temp['nBondsTimes'].fillna(0, inplace=True)
bonds = bonds.merge(temp, how='left', on='IsinIdx')
print(bonds.shape)
del temp; gc.collect()
bonds.head()

(27411, 22)


,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,IssuedAmount,CouponType,Maturity_Time,Issue_Year,Issue_Month,Issue_Week_of_Month,Maturity_Year,Maturity_Month,Maturity_Week_of_Month,nBondsTimes
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,20.943206,STEP CPN,33.109589,2005,11,4,2038,12,4,26.0
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,22.312723,FIXED,28.106849,2005,11,4,2033,12,4,469.0
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,23.432146,FIXED,28.106849,2005,11,4,2033,12,4,11.0
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,5,Government,...,22.716615,FIXED,10.008219,2007,4,3,2017,4,3,1189.0
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,21.841175,FLOATING,12.621918,2010,2,4,2022,10,1,3.0


In [27]:
# No unique customer buys a bond
feat = ['CustomerIdx', 'IsinIdx']
temp = full[full.CustomerInterest == 1][feat].drop_duplicates().groupby(['IsinIdx']).size().reset_index()
temp.columns = ['IsinIdx', 'nBondsUniqueCust']
temp['nBondsUniqueCust'].fillna(0, inplace=True)
bonds = bonds.merge(temp, how='left', on='IsinIdx')
print(bonds.shape)
del temp; gc.collect()
bonds.head()

(27411, 23)


,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,CouponType,Maturity_Time,Issue_Year,Issue_Month,Issue_Week_of_Month,Maturity_Year,Maturity_Month,Maturity_Week_of_Month,nBondsTimes,nBondsUniqueCust
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,STEP CPN,33.109589,2005,11,4,2038,12,4,26.0,14.0
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,FIXED,28.106849,2005,11,4,2033,12,4,469.0,44.0
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,FIXED,28.106849,2005,11,4,2033,12,4,11.0,5.0
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,5,Government,...,FIXED,10.008219,2007,4,3,2017,4,3,1189.0,117.0
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,FLOATING,12.621918,2010,2,4,2022,10,1,3.0,1.0


In [29]:
# Bond reBuying ratio
bonds['reBuying_ratio'] = bonds.nBondsTimes/bonds.nBondsUniqueCust
bonds['reBuying_ratio'].fillna(0, inplace=True)
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,Maturity_Time,Issue_Year,Issue_Month,Issue_Week_of_Month,Maturity_Year,Maturity_Month,Maturity_Week_of_Month,nBondsTimes,nBondsUniqueCust,reBuying_ratio
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,33.109589,2005,11,4,2038,12,4,26.0,14.0,1.857143
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,28.106849,2005,11,4,2033,12,4,469.0,44.0,10.659091
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,28.106849,2005,11,4,2033,12,4,11.0,5.0,2.200000
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,5,Government,...,10.008219,2007,4,3,2017,4,3,1189.0,117.0,10.162393
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,12.621918,2010,2,4,2022,10,1,3.0,1.0,3.000000


In [30]:
bonds.nBondsTimes.describe()

count    27169.000000
mean        78.827377
std        109.699823
min          1.000000
25%         13.000000
50%         43.000000
75%        102.000000
max       2279.000000
Name: nBondsTimes, dtype: float64

In [31]:
bonds.nBondsTimes.sum()

2141661.0

In [32]:
# Probability of bond being bought: nTimes/nbondsTimes*100
bonds['bond_probability'] = (bonds.nBondsTimes/2141661.0)*100
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,Issue_Year,Issue_Month,Issue_Week_of_Month,Maturity_Year,Maturity_Month,Maturity_Week_of_Month,nBondsTimes,nBondsUniqueCust,reBuying_ratio,bond_probability
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,2005,11,4,2038,12,4,26.0,14.0,1.857143,0.001214
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,2005,11,4,2033,12,4,469.0,44.0,10.659091,0.021899
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,2005,11,4,2033,12,4,11.0,5.0,2.200000,0.000514
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,5,Government,...,2007,4,3,2017,4,3,1189.0,117.0,10.162393,0.055518
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,2010,2,4,2022,10,1,3.0,1.0,3.000000,0.000140


In [48]:
# nBondTimes for week 0,1,2,3,4,5
feat = ['CustomerIdx', 'IsinIdx', 'Week_of_Month']
temp = full[full.CustomerInterest == 1][feat].groupby(['IsinIdx', 'Week_of_Month']).size().reset_index()
temp.columns = ['IsinIdx', 'Week_of_Month', 'nBondsTimes']
temp['nBondsTimes'].fillna(0, inplace=True)
temp = temp.pivot(index='IsinIdx', columns='Week_of_Month', values='nBondsTimes').reset_index()
temp.columns = ['IsinIdx', 'nBondTimes_Week0', 'nBondTimes_Week1', 'nBondTimes_Week2', 'nBondTimes_Week3', 
                'nBondTimes_Week4', 'nBondTimes_Week5']
temp.fillna(0, inplace=True)
bonds = bonds.merge(temp, how='left')
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,nBondsTimes,nBondsUniqueCust,reBuying_ratio,bond_probability,nBondTimes_Week0,nBondTimes_Week1,nBondTimes_Week2,nBondTimes_Week3,nBondTimes_Week4,nBondTimes_Week5
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,26.0,14.0,1.857143,0.001214,1.0,4.0,7.0,2.0,12.0,0.0
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,469.0,44.0,10.659091,0.021899,35.0,106.0,99.0,115.0,103.0,11.0
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,11.0,5.0,2.200000,0.000514,0.0,0.0,4.0,0.0,7.0,0.0
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,5,Government,...,1189.0,117.0,10.162393,0.055518,109.0,287.0,293.0,251.0,226.0,23.0
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,3.0,1.0,3.000000,0.000140,0.0,2.0,1.0,0.0,0.0,0.0


In [53]:
# Probability of bond for week 1,2,3,4
bonds['bond_probability_week0'] = (bonds.nBondTimes_Week0/bonds.nBondsTimes)*bonds.bond_probability
bonds['bond_probability_week1'] = (bonds.nBondTimes_Week1/bonds.nBondsTimes)*bonds.bond_probability
bonds['bond_probability_week2'] = (bonds.nBondTimes_Week2/bonds.nBondsTimes)*bonds.bond_probability
bonds['bond_probability_week3'] = (bonds.nBondTimes_Week3/bonds.nBondsTimes)*bonds.bond_probability
bonds['bond_probability_week4'] = (bonds.nBondTimes_Week4/bonds.nBondsTimes)*bonds.bond_probability
bonds['bond_probability_week5'] = (bonds.nBondTimes_Week5/bonds.nBondsTimes)*bonds.bond_probability
bonds.head()

,IsinIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,...,nBondTimes_Week2,nBondTimes_Week3,nBondTimes_Week4,nBondTimes_Week5,bond_probability_week0,bond_probability_week1,bond_probability_week2,bond_probability_week3,bond_probability_week4,bond_probability_week5
0,0,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,7.0,2.0,12.0,0.0,0.000047,0.000187,0.000327,0.000093,0.000560,0.000000
1,1,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,99.0,115.0,103.0,11.0,0.001634,0.004949,0.004623,0.005370,0.004809,0.000514
2,2,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,4.0,0.0,7.0,0.0,0.000000,0.000000,0.000187,0.000000,0.000327,0.000000
3,3,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,5,Government,...,293.0,251.0,226.0,23.0,0.005090,0.013401,0.013681,0.011720,0.010553,0.001074
4,4,GOV,ARS,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,10,Government,...,1.0,0.0,0.0,0.0,0.000000,0.000093,0.000047,0.000000,0.000000,0.000000


In [33]:
# Total number of time bond is bought/sold
feat = ['CustomerIdx', 'IsinIdx', 'Week_of_Month']
temp = full[full.CustomerInterest == 1][feat].groupby(['IsinIdx', 'Week_of_Month']).size().reset_index()
temp.columns = ['IsinIdx', 'nBondsTimes']
temp['nBondsTimes'].fillna(0, inplace=True)
bonds = bonds.merge(temp, how='left', on='IsinIdx')
print(bonds.shape)
del temp; gc.collect()
bonds.head()

4    1406457
2    1057124
1    1011926
3     977730
0     311413
5     169699
Name: Week_of_Month, dtype: int64

In [54]:
# No of times this bond is bought last week
feat = ['CustomerIdx', 'IsinIdx', 'WeekID', 'Prev_WeekID']
temp = full[full.CustomerInterest != 0][feat].groupby(['IsinIdx', 'WeekID', 'Prev_WeekID']).size().reset_index()
temp.columns = ['IsinIdx', 'WeekID', 'Prev_WeekID', 'bond_nTimes_lastWeek']
temp = temp[['IsinIdx', 'Prev_WeekID']].merge(temp[['IsinIdx', 'WeekID', 'bond_nTimes_lastWeek']], left_on=['IsinIdx', 'Prev_WeekID'], right_on=['IsinIdx', 'WeekID'])
temp.drop('WeekID', axis=1, inplace=True)
full = full.merge(temp, how='left')
full['bond_nTimes_lastWeek'].fillna(0, inplace=True)
del temp; gc.collect()
print(full.shape)
full.head()

(4934349, 22)


,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,PredictionIdx,Prev_WeekID,TimeHash,WeekID,Week_of_Month,...,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread,bond_nTimes_lastWeek
0,Sell,2789,1.0,8478,12,trade,56,2016121,57,1,...,86.4376,8.8726,6.7894,85.677,8.801,6.702,86.812,9.017,6.946,0.0
1,Sell,2398,1.0,9452,12,trade,56,2016121,57,1,...,98.3128,5.6364,3.5346,97.062,5.331,3.162,100.281,5.832,3.743,1.0
2,Sell,2398,1.0,12036,12,trade,56,2016121,57,1,...,102.8648,4.0736,2.4798,102.125,3.973,2.360,103.188,4.311,2.717,1.0
3,Buy,2574,1.0,6921,12,trade,56,2016121,57,1,...,100.3772,6.9206,4.9414,99.375,6.852,4.842,100.750,7.099,5.017,1.0
4,Sell,2574,1.0,7762,12,trade,56,2016121,57,1,...,100.1488,1.1422,0.1536,100.129,1.048,0.077,100.168,1.213,0.198,1.0


In [56]:
full[full.WeekID ==141].head()

,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,PredictionIdx,Prev_WeekID,TimeHash,WeekID,Week_of_Month,...,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread,bond_nTimes_lastWeek
4449591,Buy,1856,NaN,13323,4,a1e0d80784,140,2018044,141,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4449592,Buy,1856,NaN,9230,4,c2cc6cc2a8,140,2018044,141,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4449593,Buy,1780,NaN,9157,4,a8e94f6344,140,2018044,141,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4449594,Buy,2129,NaN,9131,4,758bae1e35,140,2018044,141,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4449595,Buy,1758,NaN,7151,4,02ab378ee8,140,2018044,141,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [62]:
full.shape

(4576336, 22)

In [61]:
full.drop_duplicates(inplace=True)

In [64]:
full.drop_duplicates(['BuySell', 'CustomerIdx', 'IsinIdx', 'WeekID']).shape

(4576336, 22)

In [71]:
# Shifting price by 1 week
lfeat = ['BuySell', 'CustomerIdx', 'CustomerInterest', 'IsinIdx', 'Month', 'PredictionIdx', 'Prev_WeekID', 
         'TimeHash', 'WeekID', 'Week_of_Month', 'Year', 'Cust_Previous_WeekID', 'bond_nTimes_lastWeek']
rfeat = ['BuySell', 'CustomerIdx', 'IsinIdx', 'WeekID', 'Avg_Price', 'Avg_Yield', 'Avg_ZSpread',
         'Min_Price', 'Min_Yield', 'Min_ZSpread', 'Max_Price', 'Max_Yield', 'Max_ZSpread']
lon = ['BuySell', 'CustomerIdx', 'IsinIdx', 'Prev_WeekID']
ron = ['BuySell', 'CustomerIdx', 'IsinIdx', 'WeekID']

full = full[lfeat].merge(full[rfeat], how='left', left_on=lon, right_on=ron)
full.drop(['WeekID_y'], axis=1, inplace=True)
full.rename(columns = {'WeekID_x':'WeekID'},inplace = True)
full.head()

,BuySell,CustomerIdx,CustomerInterest,IsinIdx,Month,PredictionIdx,Prev_WeekID,TimeHash,WeekID,Week_of_Month,...,bond_nTimes_lastWeek,Avg_Price,Avg_Yield,Avg_ZSpread,Min_Price,Min_Yield,Min_ZSpread,Max_Price,Max_Yield,Max_ZSpread
0,Sell,2789,1.0,8478,12,trade,56,2016121,57,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sell,2398,1.0,9452,12,trade,56,2016121,57,1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sell,2398,1.0,12036,12,trade,56,2016121,57,1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Buy,2574,1.0,6921,12,trade,56,2016121,57,1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sell,2574,1.0,7762,12,trade,56,2016121,57,1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
full.isnull().sum()

BuySell                 2307930
CustomerIdx                   0
CustomerInterest         484758
IsinIdx                       0
Month                         0
PredictionIdx                 0
Prev_WeekID                   0
TimeHash                      0
WeekID                        0
Week_of_Month                 0
Year                          0
Cust_Previous_WeekID          0
bond_nTimes_lastWeek          0
Avg_Price               4183044
Avg_Yield               4183044
Avg_ZSpread             4183044
Min_Price               4183044
Min_Yield               4183044
Min_ZSpread             4183044
Max_Price               4183044
Max_Yield               4183044
Max_ZSpread             4183044
dtype: int64

In [73]:
full.shape

(4576336, 22)

In [74]:
full.columns

Index(['BuySell', 'CustomerIdx', 'CustomerInterest', 'IsinIdx', 'Month',
       'PredictionIdx', 'Prev_WeekID', 'TimeHash', 'WeekID', 'Week_of_Month',
       'Year', 'Cust_Previous_WeekID', 'bond_nTimes_lastWeek', 'Avg_Price',
       'Avg_Yield', 'Avg_ZSpread', 'Min_Price', 'Min_Yield', 'Min_ZSpread',
       'Max_Price', 'Max_Yield', 'Max_ZSpread'],
      dtype='object')

In [79]:
full = reduce_mem_usage(full)

Memory usage of dataframe is 803.04 MB
Memory usage after optimization is: 331.69 MB
Decreased by 58.7%


In [80]:
feat = ['BuySell', 'CustomerIdx', 'IsinIdx', 'WeekID', 'bond_nTimes_lastWeek', 'Avg_Price',
       'Avg_Yield', 'Avg_ZSpread', 'Min_Price', 'Min_Yield', 'Min_ZSpread', 'Max_Price', 'Max_Yield', 'Max_ZSpread']
full[feat].to_csv(path_input+'trade_challenge_bond_feat.csv', index=False)

In [76]:
bonds.columns

Index(['IsinIdx', 'Seniority', 'Currency', 'ActivityGroup', 'Region',
       'Activity', 'RiskCaptain', 'Owner', 'CompositeRating', 'IndustrySector',
       'IndustrySubgroup', 'MarketIssue', 'IssuedAmount', 'CouponType',
       'Maturity_Time', 'Issue_Year', 'Issue_Month', 'Issue_Week_of_Month',
       'Maturity_Year', 'Maturity_Month', 'Maturity_Week_of_Month',
       'nBondsTimes', 'nBondsUniqueCust', 'reBuying_ratio', 'bond_probability',
       'nBondTimes_Week0', 'nBondTimes_Week1', 'nBondTimes_Week2',
       'nBondTimes_Week3', 'nBondTimes_Week4', 'nBondTimes_Week5',
       'bond_probability_week0', 'bond_probability_week1',
       'bond_probability_week2', 'bond_probability_week3',
       'bond_probability_week4', 'bond_probability_week5'],
      dtype='object')

In [81]:
bonds = reduce_mem_usage(bonds)

Memory usage of dataframe is 7.95 MB
Memory usage after optimization is: 3.74 MB
Decreased by 53.0%


In [82]:
bonds.to_csv(path_input+'bonds_updated.csv', index=False)

In [ ]:
# Bonds average issue amount
# Only one user count ratio: Ratio of times only this bond was in transaction by a user
# Bonds together with: No of other bonds bought with this bond on an average 